In [1]:
!pip install -q grape -U
!pip install -q plot_keras_history seedir silence_tensorflow
# !pip install -q tsnecuda==3.0.0+cu110 -f https://tsnecuda.isx.ai/tsnecuda_stable.html --no-dependencies
!pip install -q faiss

     |████████████████████████████████| 97 kB 3.3 MB/s 
     |████████████████████████████████| 99 kB 8.8 MB/s 
     |████████████████████████████████| 112 kB 5.4 MB/s 
     |████████████████████████████████| 174 kB 42.9 MB/s 


In [2]:
!wget https://kg-hub.berkeleybop.io/kg-ontoml/20220304/KG-OntoML.tar.gz

--2022-03-04 22:28:01--  https://kg-hub.berkeleybop.io/kg-ontoml/20220304/KG-OntoML.tar.gz
Resolving kg-hub.berkeleybop.io (kg-hub.berkeleybop.io)... 108.156.91.115, 108.156.91.16, 108.156.91.114, ...
Connecting to kg-hub.berkeleybop.io (kg-hub.berkeleybop.io)|108.156.91.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44235554 (42M) [application/gzip]
Saving to: ‘KG-OntoML.tar.gz.1’

KG-OntoML.tar.gz.1  100%[===================>]  42.19M  78.9MB/s    in 0.5s    

2022-03-04 22:28:02 (78.9 MB/s) - ‘KG-OntoML.tar.gz.1’ saved [44235554/44235554]



In [3]:
!tar -xvzf KG-OntoML.tar.gz

merged-kg_nodes.tsv
merged-kg_edges.tsv


In [4]:
!head merged-kg_nodes.tsv

id	category	name	description	xref	provided_by	synonym	0000052	0000061	0000114	0000178	0000184	0000231	0000233	0000234	0000386	0000399	0000424	0000425	0000426	0000589	0000905	0006012	0037192	0100001	9000002	:http://attempto.ifi.uzh.ch/ace_lexicon#TV_pl	:http://attempto.ifi.uzh.ch/ace_lexicon#TV_sg	:http://attempto.ifi.uzh.ch/ace_lexicon#TV_vbg	:http://geneontology.org/formats/oboInOwl#created_by	:http://purl.obolibrary.org/obo/chebi/charge	:http://purl.obolibrary.org/obo/chebi/formula	:http://purl.obolibrary.org/obo/chebi/inchi	:http://purl.obolibrary.org/obo/chebi/inchikey	:http://purl.obolibrary.org/obo/chebi/mass	:http://purl.obolibrary.org/obo/chebi/monoisotopicmass	:http://purl.obolibrary.org/obo/chebi/smiles	:http://purl.org/spar/cito/citesAsAuthority	:http://usefulinc.com/ns/doap#revision	:http://www.ebi.ac.uk/cellline/definition	:http://www.ebi.ac.uk/cellline/definition_editor	:http://www.ebi.ac.uk/efo/alternative_term	:http://www.ebi.ac.uk/efo/definition	:http://www.ebi.ac.uk/e

In [5]:
!head merged-kg_edges.tsv

id	subject	predicate	object	category	relation	knowledge_source	logical_interpretation
OBO:FBbt_00004566-biolink:subclass_of-OBO:FBbt_00004554	OBO:FBbt_00004566	biolink:subclass_of	OBO:FBbt_00004554		rdfs:subClassOf	Graph	
urn:uuid:a564b8a3-6fee-45f8-8cc8-d0d6093c6c64	OBO:FBbt_00004566	biolink:develops_from	OBO:FBbt_00001781	biolink:Association	RO:0002202	Graph	owlstar:AllSomeInterpretation
urn:uuid:c1ef3711-892c-4470-b9b0-062fc0c95fda	OBO:FBbt_00004566	biolink:part_of	OBO:FBbt_00004565	biolink:Association	BFO:0000050	Graph	owlstar:AllSomeInterpretation
OBO:FBbt_00004554-biolink:subclass_of-OBO:FBbt_00004475	OBO:FBbt_00004554	biolink:subclass_of	OBO:FBbt_00004475		rdfs:subClassOf	Graph	
urn:uuid:1f6a4aa4-abee-4335-acef-cab6be117d77	OBO:FBbt_00004554	biolink:part_of	OBO:FBbt_00004553	biolink:Association	BFO:0000050	Graph	owlstar:AllSomeInterpretation
XPO:0136009-biolink:subclass_of-XPO:0135498	XPO:0136009	biolink:subclass_of	XPO:0135498		rdfs:subClassOf	Graph	
XPO:0136009-biolink:subclas

In [6]:
# In order to disable often useless TensorFlow warnings
import silence_tensorflow.auto

In [7]:
import ensmallen
from ensmallen import Graph
import embiggen

In [8]:
g= Graph.from_csv(
  directed=False,
  node_path='merged-kg_nodes.tsv',
  edge_path='merged-kg_edges.tsv',
  verbose=True,
  nodes_column='id',
  node_list_node_types_column='category',
  default_node_type='biolink:NamedThing',
  sources_column='subject',
  destinations_column='object',
  default_edge_type='biolink:related_to'
)

In [9]:
g

<p>The undirected graph has 511960 homogeneous nodes and 766323 homogeneous edges.</p><h3>RAM requirements</h3><p>The RAM requirements for the nodes and edges data structures are 129.83MB and 3.73MB respectively.</p><h3>Degree centrality</h3><p>The minimum node degree is 0, the maximum node degree is 21699, the mode degree is 1, the mean degree is 2.99 and the node degree median is 1.</p><p>The nodes with highest degree centrality are: <a href='http://purl.obolibrary.org/obo/CLO_0000374' target='_blank' title='Go to Cell Line Ontology to get more informations about CLO:0000374'>CLO:0000374</a> (degree 21699 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), GOP:chebi_ph7_3 (degree 11295 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=species' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:species'>NCBITaxon:species</a> (degree 5014 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/IAO_0000227' target='_blank' title='Go to Information Artifact Ontology to get more informations about IAO:0000227'>IAO:0000227</a> (degree 4510 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>) and <a href='http://purl.obolibrary.org/obo/mondo#ordo_disease' target='_blank' title='Go to Open Biomedical Ontology to get more informations about OBO:mondo#ordo_disease'>OBO:mondo#ordo_disease</a> (degree 3981 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>).</p><h3>Disconnected nodes</h3><p>Disconnected nodes are nodes that are not connected to any other node.The graph contains 46924 disconnected nodes.</p><h4>Singleton nodes</h4><p>Singleton nodes are nodes with no edge to other nodes nor selfloops. The graph contains 46922 singleton nodes, which are <a href='http://purl.obolibrary.org/obo/GENO_0000634' target='_blank' title='Go to Genotype Ontology to get more informations about GENO:0000634'>GENO:0000634</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/GOREL_0002005' target='_blank' title='Go to Gene Ontology Relations to get more informations about GOREL:0002005'>GOREL:0002005</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/CLO_0000167' target='_blank' title='Go to Cell Line Ontology to get more informations about CLO:0000167'>CLO:0000167</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://www.obofoundry.org/ro/ro.owl#contained_in' target='_blank' title='Go to website to get more informations about http:&#x2f;&#x2f;www.obofoundry.org&#x2f;ro&#x2f;ro.owl#contained_in'>http:&#x2f;&#x2f;www.obofoundry.org&#x2f;ro&#x2f;ro.owl#contained_in</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get mo

In [10]:
from embiggen.pipelines import compute_node_embedding
from plot_keras_history import plot_history

node_embedding_method_name = "SkipGram"

first_order_rw_node_embedding, training_history = compute_node_embedding(
    g,
    node_embedding_method_name=node_embedding_method_name,
)

plot_history(
    training_history,
    title="First-order random walk based {} model applied to graph {}".format(
        node_embedding_method_name,
        g.get_name()
    )
)

0epoch [00:00, ?epoch/s]

KeyboardInterrupt: ignored